In [8]:
def load_data_set():
    data_set = [['1', '3', '4'], ['2', '3',"5"], ['1', '2','3','5'],
            ['2', '5']]
    return data_set
data = load_data_set()
data
# https://www.cnblogs.com/pinard/p/6293298.html 参考这篇文章实现

[['1', '3', '4'], ['2', '3', '5'], ['1', '2', '3', '5'], ['2', '5']]

In [22]:
from collections import Counter
class Apriori:
    # k 最大项数，min_sup 最小支持度，data数据
    def __init__(self,k,min_sup,data):
        self.k=k
        self.min=min_sup
        self.data=data
        self.data_num=len(data)
        self.min_num=int(self.data_num*min_sup)
        # 把列表数据拼接成字符串好统计支持度
    def generate_str_list(self):
        self.str_list=[]
        for li in self.data:
            self.str_list.append(self.listToStr(sorted(li)))   
    def fit(self):
        self.generate_str_list()
        # k=1
        joinL= self.generateL_1()
        for i in range(self.k-1):
            next_C=self.generate_C(joinL,i+2)
           
                
            joinL=self.generate_L(next_C)
            self.joinL=joinL
            if len(joinL)==0:
                
                return
            self.print_sup(joinL)
            self.print_confidence(joinL)
    # 输出支持度        
    def print_sup(self,joinL):
        print_content={}
        print("*"*50)
        for key in joinL:
            print_content[key]=str(joinL[key] *100.0/self.data_num) +"%"
        print('支持度:',print_content)    
     # 输出置信度   
    def print_confidence(self,joinL):
        print_content={}
        print("*"*50)
        for key in joinL:
            key_1=key[:-1]
            key_2=key[-1]
            count_1=0
            count=0
            for str_ in self.str_list:
                if self.str_in_str(key_1,str_):
                    count_1+=1
                if self.str_in_str(key,str_):
                    count+=1
            print_content[key_1+"---->"+key_2]= str(100*(count /self.data_num )*1./ (count_1 /self.data_num )) +"%"  
            
        print('置信度:',print_content)  
    
    # 生成 L1
    def generateL_1(self):
        
        self.L_1={}
        split_list = [i for k in self.data for i in k]
        counter =Counter(split_list)
        result_dict=dict(counter)
        for key  in result_dict:
            value=result_dict[key]
            if value >=self.min_num:
                self.L_1[key]=value       
        return self.L_1
    # 生成 候选频繁项C
    def generate_C(self,join,k):
        next_C=set()
        dict_c={}
        for key_1 in join:
            for key_2 in self.L_1:
                if key_1 != key_2 and len(set(key_1+key_2)) >=k:
                    next_C.add(self.listToStr(sorted(key_1+key_2)))            
        for next_c in next_C:
            count=0
            for key in self.str_list:
                if self.str_in_str(next_c,key):
                    count+=1
                dict_c[next_c]=count        
        return dict_c
    # 生成频繁项L
    def generate_L(self,next_C):
        
        joinL={}
        for key in next_C:
            if next_C[key]>=self.min_num:
                joinL[key]=next_C[key]
        return joinL
    # 列表变字符串
    def listToStr(self,list_str):
        str_list=""
        for c in list_str:
            str_list+=c;
        return str_list
    # 判断字符串 str_1 是否在字符串2里
    def str_in_str(self,str_1,str_2):
        if str_1 ==str_2:
            return True
        if len(str_1)> len(str_2):
            return False
        str_list_1=sorted(str_1)
        str_list_2=sorted(str_2)
        count=0
        for c_1 in str_list_1:

            for c_2 in str_list_2:
                if c_1 == c_2:
                    count+=1
                    break
        if len(str_list_1)==count:
            return True
        else :
            return False
 


In [23]:
model= Apriori(4,0.5,data)
model.fit()

**************************************************
支持度: {'23': '50.0%', '35': '50.0%', '13': '50.0%', '25': '75.0%'}
**************************************************
置信度: {'2---->3': '66.66666666666667%', '3---->5': '66.66666666666667%', '1---->3': '100.0%', '2---->5': '100.0%'}
**************************************************
支持度: {'235': '50.0%'}
**************************************************
置信度: {'23---->5': '100.0%'}


'23'

134
235
1235
25


['134', '235', '1235', '25']


False

In [96]:
c=0
c+=1
c

1

In [127]:
def str_in_str(str_1,str_2):
    if str_1 ==str_2:
        return True
    if len(str_1)> len(str_2):
        return False
    str_list_1=sorted(str_1)
    str_list_2=sorted(str_2)
    count=0
    for c_1 in str_list_1:

        for c_2 in str_list_2:
            if c_1 == c_2:
                count+=1
                break
    if len(str_list_1)==count:
        return True
    else :
        return False
        

In [128]:
str_in_str("2212","221")

False

In [143]:
 model.generateL_1()

{'1': 2, '3': 3, '2': 3, '5': 3}

In [3]:
"123456"[:-1]

'12345'

In [4]:
"123456"[-1]

'6'